In [1]:
from openai import OpenAI
import pandas as pd
from datasets import Dataset
import os
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

c:\Users\phku0\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_news = pd.read_parquet("../data/1.cryptonews.parquet")
df_absa = pd.read_parquet("../data/3b.cryptonews_absa.parquet")

In [3]:
print(df_news.shape)
df_news.tail()

(159486, 10)


,news_url,image_url,title,text,source_name,date,topics,sentiment,type,tickers
159481,https://www.fxempire.com/forecasts/article/xrp...,https://crypto.snapi.dev/images/v1/placeholder...,XRP News Today: BTC Hits $91k; Will SEC Appeal...,"XRP price drops below $2, weighed by SEC appea...",FXEmpire,"Tue, 31 Dec 2024 00:34:31 -0500","[tanalysis, regulations, pricemovement]",Negative,Article,"[BTC, XRP]"
159482,https://cryptonews.com/news/tether-moves-bitco...,https://crypto.snapi.dev/images/v1/s/i/7/17356...,Tether Moves Bitcoin Worth $780M to Reserve in...,"Tether, the issuer of the world's most widely ...",Cryptonews,"Tue, 31 Dec 2024 00:28:54 -0500","[stablecoins, pricemovement]",Positive,Article,"[BTC, USDT]"
159483,https://zycrypto.com/the-bull-run-continues-bi...,https://crypto.snapi.dev/images/v1/n/v/4/btc27...,The Bull Run Continues; Bitfinex Projects $200...,Digital asset exchange Bitfinex predicted an u...,Zycrypto,"Tue, 31 Dec 2024 00:21:59 -0500",[pricemovement],Positive,Article,[BTC]
159484,https://www.cryptopolitan.com/microstrategy-sh...,https://crypto.snapi.dev/images/v1/v/a/p/micro...,MicroStrategy shares drop 8% after year-end Bi...,MicroStrategy shares plunged more than 8% foll...,Cryptopolitan,"Tue, 31 Dec 2024 00:08:01 -0500",[],Negative,Article,[BTC]
159485,https://bitcoinist.com/bitcoin-etfs-see-end-of...,https://crypto.snapi.dev/images/v1/h/e/v/btc34...,Bitcoin ETFs See End-of-Year Bleeding With $38...,Spot Bitcoin (BTC) exchange-traded funds (ETFs...,Bitcoinist,"Tue, 31 Dec 2024 00:00:36 -0500",[pricemovement],Negative,Article,[BTC]


In [4]:
print(df_absa.shape)
df_absa.tail()

(155376, 10)


,title,text,source_name,date,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa
index,,,,,,,,,,
159481,XRP News Today: BTC Hits $91k; Will SEC Appeal...,"XRP price drops below $2, weighed by SEC appea...",FXEmpire,2024-12-31 05:34:31+00:00,0.0,0.0,0.0,0.0,0.0,0.0
159482,Tether Moves Bitcoin Worth $780M to Reserve in...,"Tether, the issuer of the world's most widely ...",Cryptonews,2024-12-31 05:28:54+00:00,0.0,0.0,0.0,0.0,0.0,0.0
159483,The Bull Run Continues; Bitfinex Projects $200...,Digital asset exchange Bitfinex predicted an u...,Zycrypto,2024-12-31 05:21:59+00:00,0.0,0.0,0.0,0.0,0.0,0.0
159484,MicroStrategy shares drop 8% after year-end Bi...,MicroStrategy shares plunged more than 8% foll...,Cryptopolitan,2024-12-31 05:08:01+00:00,0.0,0.0,0.0,0.0,0.0,0.0
159485,Bitcoin ETFs See End-of-Year Bleeding With $38...,Spot Bitcoin (BTC) exchange-traded funds (ETFs...,Bitcoinist,2024-12-31 05:00:36+00:00,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df_news = df_news[['title', 'text', 'source_name', 'date']]
df_news.head()

,title,text,source_name,date
0,2021 Bitcoin Price Predictions: Is The Massive...,As the bitcoin price hovers under the psycholo...,Forbes,"Fri, 01 Jan 2021 19:20:00 -0500"
1,Will Central Banks Hold Bitcoin in 2021?,Central banks (CB) will hold bitcoin sooner or...,BeInCrypto,"Fri, 01 Jan 2021 15:31:35 -0500"
2,"Bitcoin Is Digital Social Justice, feat. Tyron...",The podcaster and CEO of Onramp Invest discuss...,Coindesk,"Fri, 01 Jan 2021 14:15:02 -0500"
3,Bitcoin hits all-time high against gold as hav...,"BTC has hit another milestone, this time again...",Cointelegraph,"Fri, 01 Jan 2021 13:52:00 -0500"
4,"The Last Time This Indicator Flashed, Bitcoin ...",Bitcoin has been facing some turbulence as of ...,Bitcoinist,"Fri, 01 Jan 2021 13:00:00 -0500"


In [6]:
df_absa = df_absa[['tanalysis_absa', 'economy_absa', 'regulation_absa', 'technology_absa', 'adoption_absa', 'cybersecurity_absa']]
df_absa.head()

,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa
index,,,,,,
0,0.5,0.0,0.0,0.0,0.0,0.0
1,0.0,0.5,0.0,0.0,0.7,0.0
2,0.0,0.3,0.0,0.5,0.7,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,0.3,0.0,0.0,0.0,0.0,0.0


In [7]:
# merge the two dataframes on index
df = pd.concat([df_news, df_absa], axis=1)
df

,title,text,source_name,date,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa
0,2021 Bitcoin Price Predictions: Is The Massive...,As the bitcoin price hovers under the psycholo...,Forbes,"Fri, 01 Jan 2021 19:20:00 -0500",0.5,0.0,0.0,0.0,0.0,0.0
1,Will Central Banks Hold Bitcoin in 2021?,Central banks (CB) will hold bitcoin sooner or...,BeInCrypto,"Fri, 01 Jan 2021 15:31:35 -0500",0.0,0.5,0.0,0.0,0.7,0.0
2,"Bitcoin Is Digital Social Justice, feat. Tyron...",The podcaster and CEO of Onramp Invest discuss...,Coindesk,"Fri, 01 Jan 2021 14:15:02 -0500",0.0,0.3,0.0,0.5,0.7,0.0
3,Bitcoin hits all-time high against gold as hav...,"BTC has hit another milestone, this time again...",Cointelegraph,"Fri, 01 Jan 2021 13:52:00 -0500",1.0,0.0,0.0,0.0,0.0,0.0
4,"The Last Time This Indicator Flashed, Bitcoin ...",Bitcoin has been facing some turbulence as of ...,Bitcoinist,"Fri, 01 Jan 2021 13:00:00 -0500",0.3,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
159481,XRP News Today: BTC Hits $91k; Will SEC Appeal...,"XRP price drops below $2, weighed by SEC appea...",FXEmpire,"Tue, 31 Dec 2024 00:34:31 -0500",0.0,0.0,0.0,0.0,0.0,0.0
159482,Tether Moves Bitcoin Worth $780M to Reserve in...,"Tether, the issuer of the world's most widely ...",Cryptonews,"Tue, 31 Dec 2024 00:28:54 -0500",0.0,0.0,0.0,0.0,0.0,0.0
159483,The Bull Run Continues; Bitfinex Projects $200...,Digital asset exchange Bitfinex predicted an u...,Zycrypto,"Tue, 31 Dec 2024 00:21:59 -0500",0.0,0.0,0.0,0.0,0.0,0.0
159484,MicroStrategy shares drop 8% after year-end Bi...,MicroStrategy shares plunged more than 8% foll...,Cryptopolitan,"Tue, 31 Dec 2024 00:08:01 -0500",0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# print rows with NaN values or whole rows with 0 values
rows_with_nan = df[df.isnull().any(axis=1)]
rows_with_nan

,title,text,source_name,date,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa
12,VanEck makes another Bitcoin ETF run,Investment management firm VanEck has filed wi...,Modern Consensus,"Fri, 01 Jan 2021 11:07:58 -0500",NaN,NaN,NaN,NaN,NaN,NaN
16,Markets Report: Bitcoin starts 2021 with a ban...,Bitcoin begins a new year on a high in almost ...,Modern Consensus,"Fri, 01 Jan 2021 09:51:10 -0500",NaN,NaN,NaN,NaN,NaN,NaN
127,"BitPay CCO Says Bitcoin Will Hit $45,000",Jan.04 -- Bitpay Chief Commercial Officer Sonn...,Bloomberg Technology,"Mon, 04 Jan 2021 20:29:35 -0500",NaN,NaN,NaN,NaN,NaN,NaN
166,"Markets Report: Bitcoin Dives to $27,700 as a ...",Bitcoin is seeing a wild start to the week wit...,Modern Consensus,"Mon, 04 Jan 2021 09:01:49 -0500",NaN,NaN,NaN,NaN,NaN,NaN
221,"JPMorgan analysts: Bitcoin will reach $146K, b...","To do that, it will need to continue attractin...",Modern Consensus,"Tue, 05 Jan 2021 12:54:40 -0500",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
159453,5 Cryptocurrency Predictions for 2025,"When the curtain closes on 2024, investors wil...",The Motley Fool,"Tue, 31 Dec 2024 05:06:00 -0500",NaN,NaN,NaN,NaN,NaN,NaN
159454,"Bitcoin Hits Resistance Near $95,000: Investor...",Bitcoin is approaching a critical resistance z...,Cointribune,"Tue, 31 Dec 2024 05:05:00 -0500",NaN,NaN,NaN,NaN,NaN,NaN
159466,MicroStrategy Stock Plunges as Bitcoin Drops t...,MicroStrategy stock faces a steep decline foll...,InvestingCube,"Tue, 31 Dec 2024 03:36:24 -0500",NaN,NaN,NaN,NaN,NaN,NaN
159468,Ethereum Outshines Bitcoin in 2024 with a Surg...,"In 2024, Ethereum saw a significant increase i...",Cointribune,"Tue, 31 Dec 2024 03:05:00 -0500",NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Select rows where all absa columns are zeros
absa_cols = ['tanalysis_absa', 'economy_absa', 'regulation_absa', 'technology_absa', 'adoption_absa', 'cybersecurity_absa']
rows_with_all_zeros = df[(df[absa_cols] == 0).all(axis=1)]

rows_with_all_zeros

,title,text,source_name,date,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa
87,Bitcoin Rally Extends Into New Year,Jan.03 -- Bitcoin is continuing its rally into...,Bloomberg Markets and Finance,"Sun, 03 Jan 2021 18:46:20 -0500",0.0,0.0,0.0,0.0,0.0,0.0
215,Data Shows Institutions are Buying Bitcoin Abo...,Bitcoin's price action has been quite mixed as...,NewsBTC,"Tue, 05 Jan 2021 15:09:14 -0500",0.0,0.0,0.0,0.0,0.0,0.0
234,"JPMorgan Puts $146,000 Price Tag on Bitcoin","JPMorgan, one of the world's largest investmen...",Crypto Briefing,"Tue, 05 Jan 2021 09:18:32 -0500",0.0,0.0,0.0,0.0,0.0,0.0
247,The institutional herd is here: Bitcoin volume...,The institutional herd has arrived to the Bitc...,Cointelegraph,"Tue, 05 Jan 2021 06:45:00 -0500",0.0,0.0,0.0,0.0,0.0,0.0
287,Bitcoin: What are the chances it will stay abo...,With Bitcoin's price rallying across spot exch...,AMBCrypto,"Wed, 06 Jan 2021 13:30:50 -0500",0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
159481,XRP News Today: BTC Hits $91k; Will SEC Appeal...,"XRP price drops below $2, weighed by SEC appea...",FXEmpire,"Tue, 31 Dec 2024 00:34:31 -0500",0.0,0.0,0.0,0.0,0.0,0.0
159482,Tether Moves Bitcoin Worth $780M to Reserve in...,"Tether, the issuer of the world's most widely ...",Cryptonews,"Tue, 31 Dec 2024 00:28:54 -0500",0.0,0.0,0.0,0.0,0.0,0.0
159483,The Bull Run Continues; Bitfinex Projects $200...,Digital asset exchange Bitfinex predicted an u...,Zycrypto,"Tue, 31 Dec 2024 00:21:59 -0500",0.0,0.0,0.0,0.0,0.0,0.0
159484,MicroStrategy shares drop 8% after year-end Bi...,MicroStrategy shares plunged more than 8% foll...,Cryptopolitan,"Tue, 31 Dec 2024 00:08:01 -0500",0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Combine the two sets of rows (union)
df_rows_to_fix = df.loc[rows_with_nan.index.union(rows_with_all_zeros.index)]

print(df_rows_to_fix.shape)
df_rows_to_fix.head()

(11970, 10)


,title,text,source_name,date,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa
12,VanEck makes another Bitcoin ETF run,Investment management firm VanEck has filed wi...,Modern Consensus,"Fri, 01 Jan 2021 11:07:58 -0500",NaN,NaN,NaN,NaN,NaN,NaN
16,Markets Report: Bitcoin starts 2021 with a ban...,Bitcoin begins a new year on a high in almost ...,Modern Consensus,"Fri, 01 Jan 2021 09:51:10 -0500",NaN,NaN,NaN,NaN,NaN,NaN
87,Bitcoin Rally Extends Into New Year,Jan.03 -- Bitcoin is continuing its rally into...,Bloomberg Markets and Finance,"Sun, 03 Jan 2021 18:46:20 -0500",0.0,0.0,0.0,0.0,0.0,0.0
127,"BitPay CCO Says Bitcoin Will Hit $45,000",Jan.04 -- Bitpay Chief Commercial Officer Sonn...,Bloomberg Technology,"Mon, 04 Jan 2021 20:29:35 -0500",NaN,NaN,NaN,NaN,NaN,NaN
166,"Markets Report: Bitcoin Dives to $27,700 as a ...",Bitcoin is seeing a wild start to the week wit...,Modern Consensus,"Mon, 04 Jan 2021 09:01:49 -0500",NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_rows_to_fix.drop(columns=['tanalysis_absa', 'economy_absa', 'regulation_absa', 'technology_absa', 'adoption_absa', 'cybersecurity_absa'], inplace=True)

print(df_rows_to_fix.shape)
df_rows_to_fix.head()

(11970, 4)


,title,text,source_name,date
12,VanEck makes another Bitcoin ETF run,Investment management firm VanEck has filed wi...,Modern Consensus,"Fri, 01 Jan 2021 11:07:58 -0500"
16,Markets Report: Bitcoin starts 2021 with a ban...,Bitcoin begins a new year on a high in almost ...,Modern Consensus,"Fri, 01 Jan 2021 09:51:10 -0500"
87,Bitcoin Rally Extends Into New Year,Jan.03 -- Bitcoin is continuing its rally into...,Bloomberg Markets and Finance,"Sun, 03 Jan 2021 18:46:20 -0500"
127,"BitPay CCO Says Bitcoin Will Hit $45,000",Jan.04 -- Bitpay Chief Commercial Officer Sonn...,Bloomberg Technology,"Mon, 04 Jan 2021 20:29:35 -0500"
166,"Markets Report: Bitcoin Dives to $27,700 as a ...",Bitcoin is seeing a wild start to the week wit...,Modern Consensus,"Mon, 04 Jan 2021 09:01:49 -0500"


In [12]:
# subset for testing if needed
df_to_fix = df_rows_to_fix.copy()

In [ ]:
# os.environ['deepseek_api_key'] = 'YOUR_KEY'
cryptonews_api_key = os.getenv('deepseek_api_key')
client = OpenAI(api_key=cryptonews_api_key, base_url="https://api.deepseek.com")

system_prompt = """
You are an expert cryptocurrency.
Your task is to analyze crypto-related news headlines and articles to perform Aspect-Based Sentiment Analysis (ABSA).
Focus on the following aspects:

1. **price_prediction**: Sentiment related to price predictions, whale movements, large transactions, trading, technical indicators.
2. **economy**: Sentiment related to macroeconomic conditions, macroeconomic indicators.
3. **regulation**: Sentiment related to regulatory news or government policies.
4. **technology**: Sentiment related to blockchain technology, protocol upgrades, innovations.
5. **adoption**: Sentiment related to crypto adoptions, ETFs, crypto projects, business developments, partnerships, popularity.
6. **cybersecurity**: Sentiment related to cybersecurity, hacks, crime, money laundering.

For each aspect, provide a sentiment score on a scale of -1 to +1, where:
- -1 = Strongly Negative
- 0 = Neutral or Not mentioned
- +1 = Strongly Positive
- round to 1 decimal place

Output the results in JSON format with the following structure:
{
  "price_prediction": [score],
  "economy": [score],
  "regulation": [score],
  "technology": [score],
  "adoption": [score],
  "cybersecurity": [score],
}
"""

def analyze_aspects(example):
  try:
    user_prompt = f"Title: {example['title']}\nText: {example['text']}"
    messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": user_prompt}
    ]
    response = client.chat.completions.create(
      model="deepseek-chat",
      messages=messages,
      response_format={'type': 'json_object'}
    )
    return {"absa": json.loads(response.choices[0].message.content)}
  except Exception as e:
    print(f"Error processing example: {example['title']}\n{e}")
    return {"absa": None}

def analyze_aspects_parallel(df, max_workers=64):
  with open("../data/3c.deepseek_absa_fix.json", "a") as file, ThreadPoolExecutor(max_workers=max_workers) as executor:
    future_to_index = {
      executor.submit(analyze_aspects, row): idx
      for idx, row in df.iterrows()
    }
    for future in as_completed(future_to_index):
      idx = future_to_index[future]
      try:
        result = future.result()
        file.write(json.dumps({"index": idx, "absa": result["absa"]}) + "\n")
      except Exception as e:
        print(f"Error processing result: {e}")
        file.write(json.dumps({"index": idx, "absa": None}) + "\n")

analyze_aspects_parallel(df_to_fix)

-----------------------------------

In [14]:
df_output = pd.read_json("../data/3c.deepseek_absa_fix.json", lines=True)
df_output.set_index('index', inplace=True)

print(df_output.shape)
df_output.head()

(11970, 1)


,absa
index,
16,"{'price_prediction': 0.9, 'economy': 0, 'regul..."
215,"{'price_prediction': 0.7000000000000001, 'econ..."
652,"{'price_prediction': 0.30000000000000004, 'eco..."
1459,"{'price_prediction': -0.30000000000000004, 'ec..."
12,"{'price_prediction': 0, 'economy': 0, 'regulat..."


In [15]:
df_fixed = pd.merge(df_to_fix, df_output, left_index=True, right_on="index")
df_fixed.head()

,title,text,source_name,date,absa
index,,,,,
12,VanEck makes another Bitcoin ETF run,Investment management firm VanEck has filed wi...,Modern Consensus,"Fri, 01 Jan 2021 11:07:58 -0500","{'price_prediction': 0, 'economy': 0, 'regulat..."
16,Markets Report: Bitcoin starts 2021 with a ban...,Bitcoin begins a new year on a high in almost ...,Modern Consensus,"Fri, 01 Jan 2021 09:51:10 -0500","{'price_prediction': 0.9, 'economy': 0, 'regul..."
87,Bitcoin Rally Extends Into New Year,Jan.03 -- Bitcoin is continuing its rally into...,Bloomberg Markets and Finance,"Sun, 03 Jan 2021 18:46:20 -0500","{'price_prediction': 0.8, 'economy': 0, 'regul..."
127,"BitPay CCO Says Bitcoin Will Hit $45,000",Jan.04 -- Bitpay Chief Commercial Officer Sonn...,Bloomberg Technology,"Mon, 04 Jan 2021 20:29:35 -0500","{'price_prediction': 0.8, 'economy': 0, 'regul..."
166,"Markets Report: Bitcoin Dives to $27,700 as a ...",Bitcoin is seeing a wild start to the week wit...,Modern Consensus,"Mon, 04 Jan 2021 09:01:49 -0500","{'price_prediction': -0.8, 'economy': 0, 'regu..."


In [16]:
absa_columns = pd.json_normalize(df_fixed['absa'])
absa_columns = absa_columns.round(1)
absa_columns.head()

,price_prediction,economy,regulation,technology,adoption,cybersecurity
0,0.0,0.0,0.3,0.0,0.7,0.0
1,0.9,0.0,0.0,0.0,0.5,0.0
2,0.8,0.0,0.0,0.0,0.0,0.0
3,0.8,0.0,0.0,0.0,0.0,0.0
4,-0.8,0.0,0.0,0.0,0.0,0.0


In [17]:
df_fixed.loc[:, absa_columns.columns] = absa_columns.values
df_fixed.head()

,title,text,source_name,date,absa,price_prediction,economy,regulation,technology,adoption,cybersecurity
index,,,,,,,,,,,
12,VanEck makes another Bitcoin ETF run,Investment management firm VanEck has filed wi...,Modern Consensus,"Fri, 01 Jan 2021 11:07:58 -0500","{'price_prediction': 0, 'economy': 0, 'regulat...",0.0,0.0,0.3,0.0,0.7,0.0
16,Markets Report: Bitcoin starts 2021 with a ban...,Bitcoin begins a new year on a high in almost ...,Modern Consensus,"Fri, 01 Jan 2021 09:51:10 -0500","{'price_prediction': 0.9, 'economy': 0, 'regul...",0.9,0.0,0.0,0.0,0.5,0.0
87,Bitcoin Rally Extends Into New Year,Jan.03 -- Bitcoin is continuing its rally into...,Bloomberg Markets and Finance,"Sun, 03 Jan 2021 18:46:20 -0500","{'price_prediction': 0.8, 'economy': 0, 'regul...",0.8,0.0,0.0,0.0,0.0,0.0
127,"BitPay CCO Says Bitcoin Will Hit $45,000",Jan.04 -- Bitpay Chief Commercial Officer Sonn...,Bloomberg Technology,"Mon, 04 Jan 2021 20:29:35 -0500","{'price_prediction': 0.8, 'economy': 0, 'regul...",0.8,0.0,0.0,0.0,0.0,0.0
166,"Markets Report: Bitcoin Dives to $27,700 as a ...",Bitcoin is seeing a wild start to the week wit...,Modern Consensus,"Mon, 04 Jan 2021 09:01:49 -0500","{'price_prediction': -0.8, 'economy': 0, 'regu...",-0.8,0.0,0.0,0.0,0.0,0.0


In [18]:
df_fixed.drop(columns=['absa'], inplace=True)

In [19]:
df_fixed.rename(columns={'price_prediction': 'tanalysis_absa',
                   'economy': 'economy_absa',
                   'regulation': 'regulation_absa',
                   'technology': 'technology_absa',
                   'adoption': 'adoption_absa',
                   'cybersecurity': 'cybersecurity_absa'}, inplace=True)

In [20]:
df_fixed.head()

,title,text,source_name,date,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa
index,,,,,,,,,,
12,VanEck makes another Bitcoin ETF run,Investment management firm VanEck has filed wi...,Modern Consensus,"Fri, 01 Jan 2021 11:07:58 -0500",0.0,0.0,0.3,0.0,0.7,0.0
16,Markets Report: Bitcoin starts 2021 with a ban...,Bitcoin begins a new year on a high in almost ...,Modern Consensus,"Fri, 01 Jan 2021 09:51:10 -0500",0.9,0.0,0.0,0.0,0.5,0.0
87,Bitcoin Rally Extends Into New Year,Jan.03 -- Bitcoin is continuing its rally into...,Bloomberg Markets and Finance,"Sun, 03 Jan 2021 18:46:20 -0500",0.8,0.0,0.0,0.0,0.0,0.0
127,"BitPay CCO Says Bitcoin Will Hit $45,000",Jan.04 -- Bitpay Chief Commercial Officer Sonn...,Bloomberg Technology,"Mon, 04 Jan 2021 20:29:35 -0500",0.8,0.0,0.0,0.0,0.0,0.0
166,"Markets Report: Bitcoin Dives to $27,700 as a ...",Bitcoin is seeing a wild start to the week wit...,Modern Consensus,"Mon, 04 Jan 2021 09:01:49 -0500",-0.8,0.0,0.0,0.0,0.0,0.0


In [21]:
absa_cols = ['tanalysis_absa', 'economy_absa', 'regulation_absa', 'technology_absa', 'adoption_absa', 'cybersecurity_absa']
for col in absa_cols:
  df_fixed[col] = pd.to_numeric(df_fixed[col], errors='coerce').fillna(0)

print(df_fixed.shape)
df_fixed.head()

(11970, 10)


,title,text,source_name,date,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa
index,,,,,,,,,,
12,VanEck makes another Bitcoin ETF run,Investment management firm VanEck has filed wi...,Modern Consensus,"Fri, 01 Jan 2021 11:07:58 -0500",0.0,0.0,0.3,0.0,0.7,0.0
16,Markets Report: Bitcoin starts 2021 with a ban...,Bitcoin begins a new year on a high in almost ...,Modern Consensus,"Fri, 01 Jan 2021 09:51:10 -0500",0.9,0.0,0.0,0.0,0.5,0.0
87,Bitcoin Rally Extends Into New Year,Jan.03 -- Bitcoin is continuing its rally into...,Bloomberg Markets and Finance,"Sun, 03 Jan 2021 18:46:20 -0500",0.8,0.0,0.0,0.0,0.0,0.0
127,"BitPay CCO Says Bitcoin Will Hit $45,000",Jan.04 -- Bitpay Chief Commercial Officer Sonn...,Bloomberg Technology,"Mon, 04 Jan 2021 20:29:35 -0500",0.8,0.0,0.0,0.0,0.0,0.0
166,"Markets Report: Bitcoin Dives to $27,700 as a ...",Bitcoin is seeing a wild start to the week wit...,Modern Consensus,"Mon, 04 Jan 2021 09:01:49 -0500",-0.8,0.0,0.0,0.0,0.0,0.0


In [22]:
print(df.shape)
df.head()

(159486, 10)


,title,text,source_name,date,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa
0,2021 Bitcoin Price Predictions: Is The Massive...,As the bitcoin price hovers under the psycholo...,Forbes,"Fri, 01 Jan 2021 19:20:00 -0500",0.5,0.0,0.0,0.0,0.0,0.0
1,Will Central Banks Hold Bitcoin in 2021?,Central banks (CB) will hold bitcoin sooner or...,BeInCrypto,"Fri, 01 Jan 2021 15:31:35 -0500",0.0,0.5,0.0,0.0,0.7,0.0
2,"Bitcoin Is Digital Social Justice, feat. Tyron...",The podcaster and CEO of Onramp Invest discuss...,Coindesk,"Fri, 01 Jan 2021 14:15:02 -0500",0.0,0.3,0.0,0.5,0.7,0.0
3,Bitcoin hits all-time high against gold as hav...,"BTC has hit another milestone, this time again...",Cointelegraph,"Fri, 01 Jan 2021 13:52:00 -0500",1.0,0.0,0.0,0.0,0.0,0.0
4,"The Last Time This Indicator Flashed, Bitcoin ...",Bitcoin has been facing some turbulence as of ...,Bitcoinist,"Fri, 01 Jan 2021 13:00:00 -0500",0.3,0.0,0.0,0.0,0.0,0.0


In [23]:
# Replace the fixed rows into the original dataframe
df_final = pd.concat([df_fixed, df[~df.index.isin(df_fixed.index)]])

print(df_final.shape)
df_final.head()

(159486, 10)


,title,text,source_name,date,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa
12,VanEck makes another Bitcoin ETF run,Investment management firm VanEck has filed wi...,Modern Consensus,"Fri, 01 Jan 2021 11:07:58 -0500",0.0,0.0,0.3,0.0,0.7,0.0
16,Markets Report: Bitcoin starts 2021 with a ban...,Bitcoin begins a new year on a high in almost ...,Modern Consensus,"Fri, 01 Jan 2021 09:51:10 -0500",0.9,0.0,0.0,0.0,0.5,0.0
87,Bitcoin Rally Extends Into New Year,Jan.03 -- Bitcoin is continuing its rally into...,Bloomberg Markets and Finance,"Sun, 03 Jan 2021 18:46:20 -0500",0.8,0.0,0.0,0.0,0.0,0.0
127,"BitPay CCO Says Bitcoin Will Hit $45,000",Jan.04 -- Bitpay Chief Commercial Officer Sonn...,Bloomberg Technology,"Mon, 04 Jan 2021 20:29:35 -0500",0.8,0.0,0.0,0.0,0.0,0.0
166,"Markets Report: Bitcoin Dives to $27,700 as a ...",Bitcoin is seeing a wild start to the week wit...,Modern Consensus,"Mon, 04 Jan 2021 09:01:49 -0500",-0.8,0.0,0.0,0.0,0.0,0.0


In [24]:
# check na
df_final.isna().sum()

title                 0
text                  0
source_name           0
date                  0
tanalysis_absa        0
economy_absa          0
regulation_absa       0
technology_absa       0
adoption_absa         0
cybersecurity_absa    0
dtype: int64

In [25]:
# print rows with NaN values or whole rows with 0 values
rows_with_nan = df_final[df_final.isnull().any(axis=1)]
rows_with_nan

,title,text,source_name,date,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa


In [26]:
df_final

,title,text,source_name,date,tanalysis_absa,economy_absa,regulation_absa,technology_absa,adoption_absa,cybersecurity_absa
12,VanEck makes another Bitcoin ETF run,Investment management firm VanEck has filed wi...,Modern Consensus,"Fri, 01 Jan 2021 11:07:58 -0500",0.0,0.0,0.3,0.0,0.7,0.0
16,Markets Report: Bitcoin starts 2021 with a ban...,Bitcoin begins a new year on a high in almost ...,Modern Consensus,"Fri, 01 Jan 2021 09:51:10 -0500",0.9,0.0,0.0,0.0,0.5,0.0
87,Bitcoin Rally Extends Into New Year,Jan.03 -- Bitcoin is continuing its rally into...,Bloomberg Markets and Finance,"Sun, 03 Jan 2021 18:46:20 -0500",0.8,0.0,0.0,0.0,0.0,0.0
127,"BitPay CCO Says Bitcoin Will Hit $45,000",Jan.04 -- Bitpay Chief Commercial Officer Sonn...,Bloomberg Technology,"Mon, 04 Jan 2021 20:29:35 -0500",0.8,0.0,0.0,0.0,0.0,0.0
166,"Markets Report: Bitcoin Dives to $27,700 as a ...",Bitcoin is seeing a wild start to the week wit...,Modern Consensus,"Mon, 04 Jan 2021 09:01:49 -0500",-0.8,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
155371,Bitcoin remains volatile despite $103k milesto...,SkyBridge Capital founder Anthony Scaramucci h...,Crypto news,"Sat, 07 Dec 2024 14:17:59 -0500",-0.3,0.0,-0.7,0.0,0.0,0.0
155372,Mt. Gox Bitcoin Payout Triggers Major Transfer...,Mt. Gox cryptocurrency exchange has triggered ...,The Currency Analytics,"Sat, 07 Dec 2024 13:48:19 -0500",0.1,0.0,0.0,0.0,0.1,0.0
155373,US Treasury Acknowledges Bitcoin as ‘Digital G...,The U.S. Treasury's Fiscal Year 2024 Q4 Report...,Bitcoin,"Sat, 07 Dec 2024 13:30:41 -0500",1.0,0.5,0.0,0.0,0.5,0.0
155374,"Can Bitcoin, Ethereum, Dogecoin, Shiba Inu or ...",The prices of cryptocurrencies have soared in ...,Benzinga,"Sat, 07 Dec 2024 13:30:17 -0500",-0.5,0.0,0.0,0.0,-0.3,0.0


In [27]:
df_final.to_parquet("../data/3d.cryptonews_absa_fixed.parquet")